In [1]:
    import importnb, pidgin, nbformat, annodize, IPython, ipywidgets, textwrap
    with pidgin.Pidgin(): import tpl

In [2]:
    def pidgin_to_voila(from_, to):
        with open(from_) as file: nb = nbformat.read(file, 4)
        links = """if 'interface' not in globals(): interface =  __import__('annodize').Interface()\n"""
        for cell in nb.cells:
            if '{'*2 in ''.join(cell['source']):
                if cell['cell_type'] == 'code':
                    if cell['outputs']:
                        template = tpl.Template(''.join(cell['source']))
                        with IPython.utils.capture.capture_output() as out:
                            IPython.display.display(template)
                        cell['outputs'][0].data.update(out.outputs[0].data)
                        cell['source'] = F"""
        {cell['source']}

            _model_{template.model_id} = tpl.Template('''{cell['source']}'''); _model_{template.model_id}"""
                        cell['source'] = __import__('pidgin').specification.transform_cell.markdown_to_python(cell['source'])

                        for k in template.unnamed_references:
                            links += F"""__import__('traitlets').dlink((interface, '{k}'), (_model_{template.model_id}, '{k}'));\n"""

        nb.cells.insert(0, nbformat.v4.new_code_cell("""    with __import__('pidgin').Pidgin():import tpl"""))
        nb.cells.append(nbformat.v4.new_code_cell(textwrap.indent(links, "    ")))
        with open(to, 'w') as file: nbformat.write(nb, file)